In [3]:
import numpy as np # Linear Algebra
import pandas as pd # data processing, CSV file I/O
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt

# New York City Taxi Fare Prediction 

describe about the city and datasets ( Tell a story)

In [4]:
nyc_train = pd.read_csv('train.csv', nrows = 6000000)
nyc_test = pd.read_csv('test.csv')

In [5]:
nyc_train.head(6)

key  fare_amount          pickup_datetime  \
0    2009-06-15 17:26:21.0000001          4.5  2009-06-15 17:26:21 UTC   
1    2010-01-05 16:52:16.0000002         16.9  2010-01-05 16:52:16 UTC   
2   2011-08-18 00:35:00.00000049          5.7  2011-08-18 00:35:00 UTC   
3    2012-04-21 04:30:42.0000001          7.7  2012-04-21 04:30:42 UTC   
4  2010-03-09 07:51:00.000000135          5.3  2010-03-09 07:51:00 UTC   
5    2011-01-06 09:50:45.0000002         12.1  2011-01-06 09:50:45 UTC   

   pickup_longitude  pickup_latitude  dropoff_longitude  dropoff_latitude  \
0        -73.844311        40.721319         -73.841610         40.712278   
1        -74.016048        40.711303         -73.979268         40.782004   
2        -73.982738        40.761270         -73.991242         40.750562   
3        -73.987130        40.733143         -73.991567         40.758092   
4        -73.968095        40.768008         -73.956655         40.783762   
5        -74.000964        40.731630         -73.972892         40.758233   

   passenger_count  
0                1  
1                1  
2                2  
3                1  
4                1  
5                1

In [6]:
# Check datatypes
nyc_train.dtypes

key                   object
fare_amount          float64
pickup_datetime       object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count        int64
dtype: object

In [7]:
nyc_train.describe()

fare_amount  pickup_longitude  pickup_latitude  dropoff_longitude  \
count  6.000000e+06      6.000000e+06     6.000000e+06       5.999961e+06   
mean   1.134049e+01     -7.251142e+01     3.992406e+01      -7.251060e+01   
std    9.810989e+00      1.280236e+01     9.022813e+00       1.285400e+01   
min   -1.000000e+02     -3.426609e+03    -3.488080e+03      -3.412653e+03   
25%    6.000000e+00     -7.399207e+01     4.073491e+01      -7.399139e+01   
50%    8.500000e+00     -7.398182e+01     4.075263e+01      -7.398016e+01   
75%    1.250000e+01     -7.396711e+01     4.076712e+01      -7.396367e+01   
max    1.273310e+03      3.439426e+03     3.344459e+03       3.457622e+03   

       dropoff_latitude  passenger_count  
count      5.999961e+06     6.000000e+06  
mean       3.992049e+01     1.684645e+00  
std        9.367604e+00     1.327794e+00  
min       -3.488080e+03     0.000000e+00  
25%        4.073404e+01     1.000000e+00  
50%        4.075316e+01     1.000000e+00  
75%        4.076811e+01     2.000000e+00  
max        3.345917e+03     2.080000e+02

- From the summary, we can see that the minimum fare amount is negative, which cannot be true and from googling we can see that the minimum fare for a taxi is \\$2.50, so we will be dropping records where the minimum fare is less than $2.50.
- New York City's minimum and maximum latitudes and longitudes lies in between -90 to 90 and -180 to 180 respectively. We will be removing rows that are out of this range.
- We will remove rows where the passenger count is zero.

## Data Cleaning 

### Removing rows where the latitudes and longitudes which doesn't lie between -90 to 90 and -180 to 180

In [26]:
nyc_train = nyc_train[((nyc_train['pickup_latitude']>-90)&(nyc_train['pickup_latitude']<90)&(nyc_train['dropoff_latitude']>-90)&(nyc_train['dropoff_latitude']<90)&(nyc_train['pickup_longitude']>-180)&(nyc_train['pickup_longitude']<180)&(nyc_train['dropoff_longitude']>-180)&(nyc_train['dropoff_longitude']<180))]

### Removing rows where the fare amount is less than \\$2.5

In [27]:
nyc_train = nyc_train[(nyc_train['fare_amount']>=2.5)]

### Removing rows where the passenger count is zero

In [28]:
nyc_train = nyc_train[(nyc_train['passenger_count']>0)]

## Data Exploration 

In [30]:
nyc_train.describe()

fare_amount  pickup_longitude  pickup_latitude  dropoff_longitude  \
count  5.978008e+06      5.978008e+06     5.978008e+06       5.978008e+06   
mean   1.135002e+01     -7.250018e+01     3.991819e+01      -7.250370e+01   
std    9.818964e+00      1.045002e+01     6.112480e+00       1.043490e+01   
min    2.500000e+00     -1.686035e+02    -7.401727e+01      -1.213912e+02   
25%    6.000000e+00     -7.399207e+01     4.073491e+01      -7.399139e+01   
50%    8.500000e+00     -7.398182e+01     4.075263e+01      -7.398016e+01   
75%    1.250000e+01     -7.396711e+01     4.076711e+01      -7.396367e+01   
max    1.273310e+03      7.393784e+01     8.974216e+01       7.394509e+01   

       dropoff_latitude  passenger_count  
count      5.978008e+06     5.978008e+06  
mean       3.992007e+01     1.690614e+00  
std        6.107386e+00     1.326188e+00  
min       -7.418194e+01     1.000000e+00  
25%        4.073404e+01     1.000000e+00  
50%        4.075316e+01     1.000000e+00  
75%        4.076811e+01     2.000000e+00  
max        8.154545e+01     2.080000e+02